In [1]:
# Import libraries
import os
from os.path import basename
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
from datetime import datetime
import tzlocal
from IPython.display import Image, HTML
import time
from time import sleep
from random import uniform
import json
import lxml
from lxml import etree
import selenium
from selenium import webdriver

In [ ]:
# API bucket scraper

# Parameters - update as required

USER_ID = os.environ("SHARECHAT_USER_ID")

PASSCODE = os.environ("SHARECHAT_PASSWORD") # inspect page > network > bucketFeed or requestType81 > headers > request payload > passcode

# Tag specific params from sharechat.com/tag > inspect ... > request payload 
bucket_dict = {
    "trending/Hindi": {
        "bucket_body": {
            "bn":"broker3","userId": USER_ID,"passCode": PASSCODE,
                        "client":"web","message":{
                            "r":"web", "f": 0, "p":"f"}},
        "api_url" : "https://restapi1.sharechat.com/requestType81"},
    "topic/whatsapp-hindi-238": {
        "bucket_body": {
            "bn":"broker3","userId": USER_ID,"passCode": PASSCODE,
                        "client":"web","message":{
                            "b":238,"allowOffline":True}},
        "api_url": "https://restapi1.sharechat.com/bucketFeed"},
    "topic/news-hindi-125": {
        "bucket_body": {
            "bn":"broker3","userId": USER_ID,"passCode": PASSCODE,
                        "client":"web","message":{
                            "b":125,"allowOffline":True}},
        "api_url": "https://restapi1.sharechat.com/bucketFeed"}}

d = os.getcwd() # Download destination

# Set timezone
local_timezone = tzlocal.get_localzone()

# Buckets to scrape
b = ["topic/news-hindi-125", "topic/whatsapp-hindi-238", "trending/Hindi"]

# Number of pages to scrape
n = 5

# Define helper functions

# Scrapes data from specified tags
def get_data(buckets, pages):
    # Create empty dataframe to collect scraped data
    df = pd.DataFrame(columns = ["link", "timestamp", "lang", 
                                   "media_type", "tag", "thumbnail"])
    print("Scraping data from Sharechat ...")
    for bucket in buckets:
        # Scrape data from each tag
        for _ in range(pages):
            url = bucket_dict[bucket]["api_url"]
            body = bucket_dict[bucket]["tag_body"]
            headers = {"content-type": "application/json", 
                           "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"} 
            response = requests.post(url, json=body, headers=headers)
            response_dict = json.loads(response.text)
            
            link, timestamp, lang, media_type = get_payload_data(response_dict)
            bucket_data = pd.DataFrame(np.column_stack([link, timestamp, lang, media_type]), 
                            columns = ["link", "timestamp", "lang", "media_type"])
            
            # Add bucket column 
            bucket_data["bucket"] = bucket
            # Add thumbnail column
            bucket_data["thumbnail"] = bucket_data["link"]
            # Add bucket data 
            df = df.append(bucket_data)  
            time.sleep(uniform(30, 35)) # random delay after each request
    df["timestamp"] = df["timestamp"].apply(lambda x: datetime.fromtimestamp(int(x), local_timezone).strftime("%d-%m-%Y, %H:%M:%S"))
    df.drop_duplicates(inplace = True)
    return df

# Gets image/video data from scraped payload 
def get_payload_data(payload_dict):
    link = []
    timestamp = []
    lang = []
    media_type = []
    for i in payload_dict["payload"]["d"]:
        if (i["t"] == "image") | (i["t"] == "video"):
            timestamp.append(i["o"])
            lang.append(i["m"])
            media_type.append(i["t"])
            if i["t"] == "image":
                link.append(i["g"])
            else:
                link.append(i["v"])
        else:
            pass # skip other content formats
    return link, timestamp, lang, media_type

# Converts links to thumbnails in html
def convert_links_to_thumbnails(df):   
    def path_to_image_html(path):
        return '<img src="'+ path + '"width="200" >' 
    image_df = df[df["media_type"] == "image"]
    pd.set_option('display.max_colwidth', -1)
    data_html = HTML(image_df.to_html(escape=False ,formatters=dict(thumbnail=path_to_image_html))) 
    return data_html

# Saves data in csv and html formats
def save_data(df, html):
    with open("sharechat_bucket_data_preview.html", "w") as f:
        f.write(html.data)
    df.drop("thumbnail", axis = 1, inplace = True)
    df.to_csv("sharechat_bucket_data.csv")

# Build API scraper
def sharechat_bucket_scraper(buckets, destination, pages):
    start_time = time.time()
    # Scrape data from each bucket
    sharechat_df = get_data(buckets, n)
    # Generate html file with image thumbnails
    sharechat_data_html = convert_links_to_thumbnails(sharechat_df)
    # Save data 
    save_data(sharechat_df, sharechat_data_html)
    print("{} posts scraped".format(len(sharechat_df)))
    print("Data saved to", destination)
    print("Time taken: %s seconds" % (time.time() - start_time))


In [ ]:
# Run bucket scraper
sharechat_bucket_scraper(b, d, n)

In [ ]:
# API scraper for tags within bucket

# Parameters for API scraper - update as required
USER_ID = 348849803 # Sharechat user id
PASSCODE = "e555de8136fb06944f7f" # inspect page > network > bucketFeed or requestType81 > headers > request payload > passcode
PATH = os.getcwd()
CHROMEDRIVER_PATH = os.path.join(PATH, "chromedriver")

# Download destination - TO UPDATE
d = "/Users/kruttikanadig/Desktop"

# Set timezone
local_timezone = tzlocal.get_localzone()

buckets = {"hindi_coronavirus": "https://sharechat.com/buckets/125?referrer=explorePage",
          "hindi_news": "https://sharechat.com/buckets/1284?referrer=bucketViewPage"}

# Helper functions for scraper
def get_tag_hashes(buckets): 
    tag_hashes = []
    for bucket in buckets:
        hashes = []
        bucket_page = buckets[bucket]
        driver = webdriver.Chrome(executable_path = CHROMEDRIVER_PATH)
        driver.get(bucket_page)
        driver.find_element_by_xpath('/html/body/div[1]/div[1]/main/div/div/div[1]').click()
        time.sleep(10)
        html = driver.page_source
        driver.quit()
        tree = etree.HTML(html)
        links = tree.xpath("//a[contains(@href, 'tag')]/@href")
        del(links[-1])
        pattern = re.compile(r'(?<=tag/).*?(?=\?)')
        for link in links:
            t_hash = re.findall(pattern, link)
            hashes.extend(t_hash)
        tag_hashes.extend(hashes)
    tag_hashes = list(set(tag_hashes))
    return tag_hashes

# Generates params for api requests
def generate_requests_dict(tag_hash, USER_ID, PASSCODE):
    requests_dict = {
    "first_request": {
        "tag_body": {
            "bn":"broker3",
            "userId": USER_ID,
            "passCode": PASSCODE, 
            "client":"web",
            "message":{
                "key": "{}".format(tag_hash), 
                "th": "{}".format(tag_hash), 
                "t": 2, 
                "allowOffline": True
                        }},
        "api_url" : "https://restapi1.sharechat.com/requestType66",
        "headers": {"content-type": "application/json", 
                    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"
                   }}, 
    "second_request": {
        "tag_body": {
            "bn":"broker3",
            "userId": USER_ID,
            "passCode": PASSCODE, 
            "client":"web",
            "message":{
                "th": "{}".format(tag_hash), 
                "allowOffline": True}},
        "api_url": "https://restapi1.sharechat.com/getViralPostsSeo",
        "headers": {"content-type": "application/json", 
                    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"
                       }}}
    return requests_dict

def get_first_payload_data(payload_dict):
    tag_name = payload_dict["payload"]["n"]
    tag_translation = payload_dict["payload"]["englishMeaning"]
    tag_genre = payload_dict["payload"]["tagGenre"]
    bucket_name = payload_dict["payload"]["bn"]
    bucket_id = payload_dict["payload"]["bi"]
    return tag_name, tag_translation, tag_genre, bucket_name, bucket_id


def get_second_payload_data(payload_dict):
    link = []
    timestamp = []
    language = []
    media_type = []
    for i in payload_dict["payload"]["d"]:
        if (i["t"] == "image") | (i["t"] == "video"):
            timestamp.append(i["o"])
            language.append(i["m"])
            media_type.append(i["t"])
            if i["t"] == "image":
                link.append(i["g"])
            else:
                link.append(i["v"])
        else:
            pass # skip other content formats
    return link, timestamp, language, media_type

# Converts links to thumbnails in html
def convert_links_to_thumbnails(df):   
    def path_to_image_html(path):
        return '<img src="'+ path + '"width="200" >' 
    image_df = df[df["media_type"] == "image"]
    pd.set_option('display.max_colwidth', -1)
    data_html = HTML(image_df.to_html(escape=False ,formatters=dict(thumbnail=path_to_image_html))) 
    return data_html

# Saves data in csv and html formats
def save_data(df, html):
    with open("sharechat_tag_data_preview.html", "w") as f:
        f.write(html.data)
    df.drop("thumbnail", axis = 1, inplace = True)
    df.to_csv("sharechat_tag_data.csv")

# Get tag data
def get_data(tag_hashes):
    # Create empty dataframe to collect scraped data
    df = pd.DataFrame(columns = ["link", "timestamp", "language", 
                                   "media_type", "tag_name", "tag_translation", "tag_genre", "bucket_name", "bucket_id", "thumbnail"])
    print("Scraping data from Sharechat ...")
    for tag_hash in tag_hashes:
        requests_dict = generate_requests_dict(tag_hash, USER_ID, PASSCODE)
        # Send API request to check tag type
        first_url = requests_dict["first_request"]["api_url"]
        first_body = requests_dict["first_request"]["tag_body"]
        first_headers = requests_dict["first_request"]["headers"]
        time.sleep(uniform(30,35)) # random time delay between requests
        first_response = requests.post(url=first_url, json=first_body, headers=first_headers)
        first_response_dict = json.loads(first_response.text)
        if (first_response_dict["payload"]["bi"] == 1284) or (first_response_dict["payload"]["tagCategory"] == "Temporary"):
            tag_name, tag_translation, tag_genre, bucket_name, bucket_id = get_first_payload_data(first_response_dict)
            second_url = requests_dict["second_request"]["api_url"]
            second_body = requests_dict["second_request"]["tag_body"]
            second_headers = requests_dict["second_request"]["headers"] 
            time.sleep(uniform(0.5,2))
            second_response = requests.post(url=second_url, json=second_body, headers=second_headers)
            second_response_dict = json.loads(second_response.text)
            link, timestamp, language, media_type = get_second_payload_data(second_response_dict)
            tag_data = pd.DataFrame(np.column_stack([link, timestamp, language, media_type]), 
                            columns = ["link", "timestamp", "language", "media_type"])
            tag_data["tag_name"] = tag_name
            tag_data["tag_translation"] = tag_translation
            tag_data["tag_genre"] = tag_genre
            tag_data["bucket_name"] = bucket_name
            tag_data["bucket_id"] = int(bucket_id)
            tag_data["thumbnail"] = tag_data["link"]
            df = df.append(tag_data)
    df["timestamp"] = df["timestamp"].apply(lambda x: datetime.fromtimestamp(int(x), local_timezone).strftime("%d-%m-%Y, %H:%M:%S"))
    df.drop_duplicates(inplace = True)
    return df

    
# Build scraper 
def sharechat_tag_scraper(buckets, destination):
    start_time = time.time()
    # Get tag hashes
    tag_hashes = get_tag_hashes(buckets)
    # Scrape data from each tag
    sharechat_df = get_data(tag_hashes)
    # Generate html file with image thumbnails
    sharechat_data_html = convert_links_to_thumbnails(sharechat_df)
    # Save data 
    save_data(sharechat_df, sharechat_data_html)
    print("{} posts scraped".format(len(sharechat_df)))
    if len(sharechat_df) > 0:
        print("Data saved to", destination)
    else:
        print("No relevant data found!")
    print("Time taken: %s seconds" % (time.time() - start_time))



In [ ]:
# Run scraper
sharechat_tag_scraper(buckets, d)

In [ ]:
# Beautiful Soup scraper

# Define scraper arguments
t = ["topic/news-hindi-125", "topic/whatsapp-hindi-238", "trending/Hindi"] # tags to scrape
d = os.getcwd() # download destination

# Define helper functions for Beautiful Soup scraper

# Scrapes data from specified tags
def get_data(tags):
    # Create empty dataframe to collect scraped data
    data = pd.DataFrame(columns = ["img_link", "timestamp", "tag", "thumbnail"])
    # Scrape data from each tag
    for tag in tags: 
        print("Scraping recent images from https://sharechat.com/"+tag+" ...")
        soup = get_parsed_page(tag) 
        img_links, timestamps = get_images_with_timestamps(soup)  
        # Save tag data as dataframe
        tag_data = pd.DataFrame(np.column_stack([img_links, timestamps]), 
                            columns = ["img_link", "timestamp"])
        # Add tag column 
        tag_data["tag"] = tag
        # Add thumbnail column
        tag_data["thumbnail"] = tag_data["img_link"]   
        # Add tag data 
        data = data.append(tag_data)
    return data

# Returns parsed web page
def get_parsed_page(tag):
    r = requests.get("https://sharechat.com/"+tag)
    c = r.content
    soup = BeautifulSoup(c, "lxml") 
    return soup

# Returns images with timestamps
def get_images_with_timestamps(soup):
    # Initialize empty lists to hold data
    img_links = []
    timestamps = []
    # Find image
    images = soup.findAll("img", {"src": re.compile(".jpg")}) 
    for image in images:
        # Add image link
        img_links.append(image["src"]) 
        # Find timestamp
        unix_ts = re.findall("\d{13}", image["src"]) 
        if (len(unix_ts) > 0): # If link contains timestamp
        # Reformat and save timestamp
            local_time = datetime.fromtimestamp(int("".join(unix_ts))/1000, local_timezone).strftime("%d:%m:%Y, %H:%M:%S")
            timestamps.append(local_time) 
        else:
            timestamps.append(None)
    return img_links, timestamps

# Adds image thumbnails for quick viewing
def convert_links_to_thumbnails(df):   
    def path_to_image_html(path):
        return '<img src="'+ path + '"width="200" >' 
    data_html = HTML(df.to_html(escape=False ,formatters=dict(thumbnail=path_to_image_html))) 
    return data_html

# Saves scraped data in csv and html formats
def save_data(df, html):
    with open("sharechat_data_html.html", "w") as f:
        f.write(html.data)
    df.drop("thumbnail", axis = 1, inplace = True)
    df.to_csv("sharechat_data.csv")
    
# Define Beautiful Soup scraper 
def sharechat_soup_scraper(tags, destination):
    # Scrape data from specified tags
    sharechat_df = get_data(tags)
    # Generate html file with image thumbnails
    sharechat_data_html = convert_links_to_thumbnails(sharechat_df)
    # Save data 
    save_data(sharechat_df, sharechat_data_html)
    print("{} images scraped".format(len(sharechat_df)))
    print("Data saved to", destination)
    

In [ ]:
# Run Beautiful Soup scraper
sharechat_soup_scraper(t, d)